In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
import math


In [2]:
MaxAccuracy = 0.00001

def Agents(SLA, ServiceTime, CallsPerHour, AHT) :
    
    Count = 0
    
    try: 
    
        if (SLA > 1): 
            SLA = 1
        BirthRate = CallsPerHour
        DeathRate = 3600 / AHT

        TrafficRate = BirthRate / DeathRate

        Erlangs = math.trunc((BirthRate * (AHT)) / 3600 + 0.5)

        if (Erlangs < 1) : 
            NoAgents = 1
        else : 
            NoAgents = math.floor(Erlangs)

        Utilisation = TrafficRate / NoAgents

        while (Utilisation >= 1) :
            NoAgents = NoAgents + 1
            Utilisation = TrafficRate / NoAgents

        MaxIterate = NoAgents * 100

        for index in range(1, MaxIterate+1) :
            Utilisation = TrafficRate / NoAgents
            if (Utilisation < 1) :
                Server = NoAgents
                C = ErlangC(Server, TrafficRate)

                SLQueued = 1 - C * math.exp((TrafficRate - Server) * ServiceTime / AHT)
                if (SLQueued < 0) : 
                    SLQueued = 0
                if (SLQueued >= SLA): 
                    Count = MaxIterate
                if (SLQueued > (1 - MaxAccuracy)): 
                    Count = MaxIterate
            if (Count != MaxIterate) : 
                NoAgents = NoAgents + 1
    except:
        NoAgents = 0
    return NoAgents


def ErlangC(Servers, Intensity) :

    try :
        if ((Servers < 0) or (Intensity < 0)) :
            return 0;
        B = ErlangB(Servers, Intensity)
        C = B / (((Intensity / Servers) * B) + (1 - (Intensity / Servers)))
    except: 
        C = 0
    return MinMax(C, 0, 1)


def ErlangB(Servers, Intensity) :
    try :
        if ((Servers < 0) or (Intensity < 0)) :
            return 0
        MaxIterate = math.trunc(Servers)
        Val = Intensity
        Last = 1
        for Count in range(MaxIterate+1) :
            B = (Val * Last) / (Count + (Val * Last))
            Last = B
    except: 
        B = 0
    return MinMax(B, 0, 1);


def MinMax(val, _min, _max) :
    result = val
    if (val < _min): 
        result = _min;
    if (val > _max): 
        result = _max;
    return result


In [3]:

%matplotlib inline
    
all_df = pd.read_csv(
    "data-1641353182031.csv",sep = ','
)

all_df = all_df.where((all_df["week"] < 42))
all_df = all_df.dropna()

def forecast(n_df,_queue):
    
    df = n_df.where(n_df['QUEUE_NAME'] == _queue)
    
    if (df.agg({"received": np.sum})["received"] < 5000): return False

    group_by_week = df.groupby("week").agg({"received": np.sum})

    livestock3 = pd.Series(group_by_week['received'])

    if (livestock3.count() < 8 ):
        return False

    fit3 = Holt(livestock3, damped_trend=True, initialization_method="estimated").fit(
        smoothing_level=0.8, smoothing_trend=0.2
    )
    fcast3 = fit3.forecast(50).rename("Additive damped trend")

    min_week = df["week"].max()-4

    four_week_data = df.where(df["week"] > min_week)

    received_group_wdate = df.groupby("week_day").agg({"received": np.sum})

    received_dis_wdate = received_group_wdate/df.agg({"received": np.sum})

    received_group_hour = four_week_data.groupby(["week_day","hour"]).agg({"received": np.sum})

    received_dis_hour = received_group_hour/four_week_data.groupby("week_day").agg({"received": np.sum})

    received_dis_hour.rename(columns = {'received':'hour_dist'}, inplace = True)

    received_dis_hour = received_dis_hour.reset_index()

    answered_group_hour = four_week_data.groupby(["week_day","hour"]).agg({"answered": np.sum})

    time_group_hour = four_week_data.groupby(["week_day","hour"]).agg({"total": np.sum})

    series1 = pd.Series(time_group_hour['total']);
    series2 = pd.Series(answered_group_hour["answered"]);

    aht_group_hour = series1.div(series2, fill_value=0).rename('aht').to_frame().reset_index()
    
    aht_group_hour = pd.merge(aht_group_hour,received_dis_wdate, on="week_day",how="left")
    
    aht_group_hour.rename(columns = {'received':'wd_dist'}, inplace = True)
    
    aht_group_hour = pd.merge(aht_group_hour, received_dis_hour, left_on=['week_day','hour'], right_on = ['week_day','hour'])
    
    aht_group_hour['global_dist'] = aht_group_hour['wd_dist'] * aht_group_hour['hour_dist'] 
    
    result = pd.DataFrame({'datetime':pd.date_range(start = '2021/12/21', periods=60*24, freq='H')})

    result['date'] = pd.to_datetime(result['datetime']).dt.date
    result['hour'] = pd.to_datetime(result['datetime']).dt.hour
    result = result.where((result["hour"] < 19) & (result["hour"] > 8) ).dropna()
    result['week_day'] = pd.to_datetime(result['datetime']).dt.dayofweek
    result = result.where(result["week_day"] < 5).dropna()
    result = pd.merge(result, aht_group_hour, left_on=['week_day','hour'] , right_on = ['week_day','hour'], how='left')
    result.fillna(value=0, inplace=True)
    
    result['weeknumber'] = result['datetime'].dt.isocalendar().week
    result['weeknumber_fake'] = result.apply(lambda row: ((row['weeknumber']+52) if row['weeknumber'] < 30 else row['weeknumber'] ) , axis = 1) 
    result['forecast_receive'] = result.apply(lambda row: (row['global_dist']*fcast3[row['weeknumber_fake']]) , axis = 1)
    result['forecast_receive'] = result.apply(lambda row: (row['forecast_receive'] if row['forecast_receive'] > 0 else 0 ) , axis = 1)
    result = result.drop(columns=['weeknumber_fake','datetime'])
    result['agent_hours'] = result.apply(lambda row: (round(Agents(0.7, 30, row['forecast_receive'], row['aht']) / (1 - 0.15))) , axis = 1)
    result['agent_hours'] = result.apply(lambda row: (row['agent_hours'] if (row['agent_hours']> 1) else 1 ), axis = 1)
    result['aht'] = result['aht'].fillna(0)
    result['aht'] = result.apply(lambda row: (round(row['aht'])) , axis = 1)
    result['queue_name'] = _queue
    result.to_csv("output/"+_queue+".csv",sep = ';') 
    return True

unprocessed_queue = []

for x in all_df["QUEUE_NAME"].unique():
    if (x == 'nan') : continues
    if (not forecast(all_df,x)): unprocessed_queue.append(x)



In [4]:
def forecast_for_less_queue(m_df, list_queue): 

    df = m_df
    boolean_series = df['QUEUE_NAME'].isin(list_queue)
    
    df = df[boolean_series]
    
    group_by_week = df.groupby("week").agg({"received": np.sum})
    
    livestock3 = pd.Series(group_by_week['received'])

    fit3 = Holt(livestock3, damped_trend=True, initialization_method="estimated").fit(
        smoothing_level=0.8, smoothing_trend=0.2
    )
    fcast3 = fit3.forecast(50).rename("Additive damped trend")

    min_week = df["week"].max()-4

    four_week_data = df.where(df["week"] > min_week)

    received_group_wdate = df.groupby("week_day").agg({"received": np.sum})

    received_dis_wdate = received_group_wdate/df.agg({"received": np.sum})
    
    received_dis_queue = df.groupby("QUEUE_NAME").agg({"received": np.sum})/df.agg({"received": np.sum})

    received_group_hour = four_week_data.groupby(["week_day","hour"]).agg({"received": np.sum})

    received_dis_hour = received_group_hour/four_week_data.groupby("week_day").agg({"received": np.sum})

    received_dis_hour.rename(columns = {'received':'hour_dist'}, inplace = True)

    received_dis_hour = received_dis_hour.reset_index()

    answered_group_hour = four_week_data.groupby(["week_day","hour"]).agg({"answered": np.sum})

    time_group_hour = four_week_data.groupby(["week_day","hour"]).agg({"total": np.sum})

    series1 = pd.Series(time_group_hour['total']);
    series2 = pd.Series(answered_group_hour["answered"]);

    aht_group_hour = series1.div(series2, fill_value=0).rename('aht').to_frame().reset_index()
    
    aht_group_hour = pd.merge(aht_group_hour,received_dis_wdate, on="week_day",how="left")
    aht_group_hour.rename(columns = {'received':'wd_dist'}, inplace = True)
    aht_group_hour = pd.merge(aht_group_hour, received_dis_hour, left_on=['week_day','hour'], right_on = ['week_day','hour'])
    aht_group_hour['global_dist'] = aht_group_hour['wd_dist'] * aht_group_hour['hour_dist'] 
    
    result = pd.DataFrame({'datetime':pd.date_range(start = '2021/12/21', periods=60*24, freq='H')})

    result['date'] = pd.to_datetime(result['datetime']).dt.date
    result['hour'] = pd.to_datetime(result['datetime']).dt.hour
    result = result.where((result["hour"] < 19) & (result["hour"] > 8) ).dropna()
    result['week_day'] = pd.to_datetime(result['datetime']).dt.dayofweek
    result = result.where(result["week_day"] < 5).dropna()
    result = pd.merge(result, aht_group_hour, left_on=['week_day','hour'] , right_on = ['week_day','hour'], how='left')
    result.fillna(value=0, inplace=True)
    
    result['weeknumber'] = result['datetime'].dt.isocalendar().week
    
    result['weeknumber_fake'] = result.apply(lambda row: ((row['weeknumber']+52) if row['weeknumber'] < 30 else row['weeknumber'] ) , axis = 1) 
    result['forecast_receive'] = result.apply(lambda row: (row['global_dist']*fcast3[row['weeknumber_fake']]) , axis = 1)
    result['forecast_receive'] = result.apply(lambda row: (row['forecast_receive'] if row['forecast_receive'] > 0 else 0 ) , axis = 1)
    result = result.drop(columns=['weeknumber_fake','datetime'])
    result['forecast_receive'] = result['forecast_receive'].fillna(0)
    result['agent_hours'] = result.apply(lambda row: (round(Agents(0.7, 30, row['forecast_receive'], row['aht']) / (1 - 0.15))), axis = 1)
    result['aht'] = result['aht'].fillna(0)
    result['aht'] = result.apply(lambda row: (round(row['aht'])) , axis = 1)
    for x in list_queue:
        queue_result = result
        queue_result['queue_name'] = x
        print(x)
        queue_result['aht'] = queue_result.apply(lambda row: (round(row['aht'])) , axis = 1)
        queue_result['forecast_receive'] = queue_result.apply(lambda row: (round(row['forecast_receive']*received_dis_queue["received"][x])) , axis = 1)
        queue_result['agent_hours'] = result.apply(lambda row: (round(row['agent_hours']*received_dis_queue["received"][x])), axis = 1)
        queue_result['agent_hours'] = result.apply(lambda row: (row['agent_hours'] if (row['agent_hours']> 1) else 1 ), axis = 1)
        name = x.replace("|", " ")
        queue_result.to_csv("output/"+name+".csv",sep = ';') 
    return True
    
forecast_for_less_queue(all_df, unprocessed_queue)


C:\Users\anhph\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'


ValueError: Cannot use heuristic method with less than 10 observations.